In [2]:
import requests
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugins.plugin_capivara import PluginCapivaraAlert

In [3]:
alerts = [
    {
        'title': "Defective product",
        'description': "I ordered a Handstückschlauch, but it came already broken",
        'severity': "Medium",
    },
    {
        'title': "Help needed during installation",
        'description': "I'm having trouble installing the Clean Reinigungsgerät, could you send someone to help me?",
        'severity': "High",
        'conversation_customer': [
            {'user': 'John', 'message': 'hello, any update?'},
        ]
    },
    {
        'title': "Order more",
        'description': "I want to buy more of the same gloves I bought last time, could you send 100 more?",
        'severity': "Low",
        'state': "closed",
        'conversation_customer': [
            {'user': 'Leonardo Benitez', 'message': 'I have placed the order as requested, you can follow the status in the order page.'},
        ]
    },
]

In [7]:
a = next(a for a in alerts)
a

{'title': 'Defective product',
 'description': 'I ordered a Handstückschlauch, but it came already broken',
 'severity': 'Medium'}

In [9]:
import requests

response = requests.post('http://capivara:80/v1/public-api-create-incidents', json=alerts)
assert response.status_code == 200
assert all([r[0] == 200 for r in response.json()])
alert_id = response.json()[0][1]
assert type(alert_id) == str
assert len(alert_id) > 0

In [12]:
response = requests.post('http://capivara:80/v1/public-api-read-incidents', json=[alert_id])
assert response.status_code == 200
assert all([r[0] == 200 for r in response.json()])
response.json()[0][1]['state']

'new'

In [13]:
plugin = PluginCapivaraAlert(
    capivara_base_url='http://capivara:80',
    alert_id=alert_id,
)
print(plugin.close_ticket(semantic_kernel_v0_to_openai_function.make_context({})))

Ticket closed successfully


In [14]:
response = requests.post('http://capivara:80/v1/public-api-read-incidents', json=[alert_id])
assert response.status_code == 200
assert all([r[0] == 200 for r in response.json()])
response.json()[0][1]['state']

'closed'